# Progetto di fine corso Algoritmi e programmazione per l'analisi dei dati 2022-2023

Il progetto prevede di rispondere a delle domande che verranno enunciate in seguito, analizzando un insieme di datests contenenti le informazioni sulle pubblicazioni scientifiche.
Lo scopo è quello di effettuare l'analisi utilizzando algoritmi sui grafi.
La libreria utilizzata per la costruzioni e metodi di utilità di grafi è [NetworkX](https://networkx.org/).

## Operazioni preliminari

Prima di iniziare l'alalisi, importiamo i principali moduli che usere:

- [NetworkX](https://networkx.org/)
- [Pandas](https://pandas.pydata.org/)
- [Numpy](https://numpy.org/)

Queste sono le librerie più usate in ambito di analisi dei dati.

In [1]:
import networkx as nx
import pandas as pd
import numpy as np
from random import choice

Essendo i dataset dei file csv, li cerchiamo all'interno della cartella del progetto, più precisamente ci aspettiamo che sia in una cartella chiamata `Data`.

Utilizzando il metodo `os.listdir(nome_path)` del modulo integrato di Python `os`, possiamo ottenere una lista con i nomi di tutti i file in quella cartella.
> Simile al comando shell `ls`.

In [2]:
import os

path = "Data"
csv_list = os.listdir(path)

A questo punto possiamo leggere i csv.
La lettura è eseguita utilizzando la libreria `pandas`.
Essa offre una conveniente struttura dati per la rappresentazione di un dataset che prende il nome di `DataFrame`. Può essere visto come una matrice, implementata tramite lista di liste.

Una lettura molto semplice richiede solo la specifica del nome del csv da leggere e il metodo `read_csv` restituirà un `DataFrame`.

Nel codice sottostante è stato eseguito un wrapper del metodo sopra citato.
Sappiamo in anticipo che colonne leggere, fatta eccezione di un unico file `out-dblp_proceedings.csv`, che ha una colonna diversa, ma con lo stesso significato al fine della nostra analisi.

L'operazione aggiuntiva è stata quella di dare un nome al Dataframe (che sarà comodo più avanti).

In [3]:
def read_csv(name:str)->pd.DataFrame:
    cols = ["id","author","title"]
    if name == "out-dblp_proceedings.csv":
        cols = ["id","editor","title"]
        
    df = pd.read_csv(
    path +"/" + name,
    delimiter=";",
    usecols=cols,
    nrows=1000
    )
    df.name = name.split(".")[0]
    df.rename(columns={"editor":"author"}, inplace=True)
    return df

## Creazione della lista dei DataFrame

Una volta che abbiamo una funzione per leggere i csv nel modo che desideriamo, ciò che vogliamo ottenere è una lista di dataframe da poter utilizzare in seguito.

Questo problema è facilmente risolvibile con un ciclo. In aggiunta, essendoci dei valori nulli, li togliamo a lettutra ultimata.

In [4]:
df_list = list()
for csv in csv_list:
    df_list.append(
        read_csv(csv)
    )
    df_list[-1].dropna(inplace = True)

### Esempio di DataFrame

Di seguito vengono riportate le prime 5 righe del primo DataFrame creato.

In [7]:
print(f"Name: {df_list[0].name}")
df_list[0].head()

Name: out-dblp_article


,id,author,title
3,4105295,Clement T. Yu|Hai He|Weiyi Meng|Yiyao Lu|Zongh...,Towards Deeper Understanding of the Search Int...
4,4106479,Fatih Gelgi|Hasan Davulcu|Srinivas Vadrevu,Information Extraction from Web Pages Using Pr...
5,4107897,Daniel A. Menascé|Vasudeva Akula,Improving the Performance of Online Auctions T...
6,4108498,Hongjian Fan|Kotagiri Ramamohanarao,Patterns Based Classifiers.
7,4108959,Bing Liu 0001|Yanhong Zhai,Extracting Web Data Using Instance-Based Learn...


## Creazione dei Grafi

Passiamo ora alla creazione del `Grafo`. Vogliamo costruire un grafo bipartito, contenente in un insieme gli `autori` e nell' altro i `titoli`.

Ciò che dobbiamo fare è dividere i vari autori di una singola pubblicazione e creare i nodi e archi desiderati.
> Gli autori sono separati da carattere `|`.

Per comodità come attributo di ogni nodo degli autori, inseriamo il conteggio degli autori che hanno partecipato a tale pubblicazione. (Questo passaggio può essere evitato perchè corrisponde al grado di tale nodo).

Appendiamo in una lista i vari grafi che abbiamo creato.

In [5]:
def create_graph(df:pd.DataFrame)->nx.Graph:
    G = nx.Graph()
    for publication_id, row in df.iterrows():
        authors = row["author"].split("|")
        title = row["title"]
        G.add_node(publication_id, bipartite = 0, title=title, authors_counter = len(authors))
        for author in authors:
            G.add_node(author, bipartite = 1)
            G.add_edge(publication_id,author)
    return G

graph_list = list()
for df in df_list:
    graph_list.append(
        create_graph(df)
    )

### Note su una possibile implementazione parallela.
La creazione del grafo di NetworkX avviene in maniera sequenziale ed è chiaramente un'operazione non vettorizzabile. Tuttavia sapendo in anticipo (come nel nostro caso) il numero di grafi da costruire, fornendo una struttura dati di output con la stessa dimensione del numero di grafi da costruire possiamo eseguire in parallelo sulla CPU la creazione di tali grafi. Naturalmente vanno fatte considerazioni aggiuntive sull'utilizzo della memoria.

La libreria più semplice da usare per gestire il parallelismo sulla CPU (senza dover instanziare altri interpreti) è probabilmente [Numba](https://numba.pydata.org/), che ci permette di avere un compilatore *just in time*.

## Pubblicazione con maggior numero di autori

Per trovare la pubblicazione con il maggior numero di autori, dobbiamo visitare il grafo nel seguente modo:
1. Selezionare tutti i nodi delle  pubblicazioni
2. All'interno di questi nodi è presente il campo `author_counter`.
3. Castando la lista ad array di `numpy`, possiamo utilizzare `argmax`.
4. Avendo l'id della pubblicazione selezioniamo il campo `titolo`.

> Se non avessimo avuto il campo `author_counter`, avremmo potuto contare il grado del nodo.

In [29]:
def get_publication_with_max_authors(G:nx.Graph)->tuple[str,int]:
    publication_ids = list(n for n, d in G.nodes(data=True) if d["bipartite"] == 0)

    authors_counter_array = np.array(
        list(map(lambda id: G.nodes[id]["authors_counter"], publication_ids))
        )

    max_authors_pubication_id = publication_ids[authors_counter_array.argmax()]

    title = G.nodes[max_authors_pubication_id]["title"]

    return (
        title,
        G.nodes[max_authors_pubication_id]['authors_counter']
    )

for idx, G in enumerate(graph_list):
    title, counter = get_publication_with_max_authors(G)
    print(f"-------------Graph: {df_list[idx].name}--------------")
    print(f"{title}\nWith {counter} authors.\n")


-------------Graph: out-dblp_article--------------
ALACRITY: Analytics-Driven Lossless Data Compression for Rapid In-Situ Indexing, Storing, and Querying.
With 13 authors.

-------------Graph: out-dblp_book--------------
A Framework of Information System Concepts (The FRISCO Report).
With 10 authors.

-------------Graph: out-dblp_incollection--------------
CoolEmAll: Models and Tools for Planning and Operating Energy Efficient Data Centres.
With 19 authors.

-------------Graph: out-dblp_inproceedings--------------
R-GMA: An Information Integration System for Grid Monitoring.
With 21 authors.

-------------Graph: out-dblp_mastersthesis--------------
Shadow Paging Is Feasible.
With 1 authors.

-------------Graph: out-dblp_phdthesis--------------
Programming Data Structures in Logic.
With 1 authors.

-------------Graph: out-dblp_proceedings--------------
Proceedings of the 2017 ACM on Conference on Information and Knowledge Management, CIKM 2017, Singapore, November 06 - 10, 2017
With 15 

In [45]:
# Autore con maggior numero di collaboratori

def get_author_with_most_collaborotors(G:nx.Graph)->tuple:
    authors = {n for n, d in G.nodes(data=True) if d["bipartite"] == 1}
    authors = list(authors)

    max = {"author": "None","collaborators":list()}
    for author in authors:
        collaborators = set()
        publication_ids = [publication_id[1] for publication_id in list(G.edges(author))]
        for publication_id in publication_ids:
            collaborators.update(
                [publication_id[1] for publication_id in list(G.edges(publication_id))]
            )
        collaborators = list(collaborators)
        collaborators.remove(author)
        if len(collaborators)  > len(max["collaborators"]):
            max["author"] = author
            max["collaborators"] = collaborators
    return(
        max['author'],
        len(max['collaborators']),
        max['collaborators']
    )

for idx, G in enumerate(graph_list):
    author, count, collaborators = get_author_with_most_collaborotors(G)
    print(f"-------------Graph: {df_list[idx].name}--------------")
    print(f"The author with most collaborators is {author}, with {count} collaborators:\n{collaborators}\n")




-------------Graph: out-dblp_article--------------
The author with most collaborators is Stéphane Bressan, with 24 collaborators:
['See-Kiong Ng', 'Shili Xiang', 'Jianneng Cao', 'Antoine Amarilli', 'Baljeet Malhotra', 'Randy Julian', 'Kian-Lee Tan', 'Mouad Hakam', 'Debabrota Basu', 'Wee-Juan Tan', 'Yi Song 0005', 'Ruiming Tang', 'Shaowen Zhou', 'Mingzhe Du', 'Thomas Kister', 'Geong Sen Poh', 'Zihong Yuan', 'Gillian Dobbie', 'Qian Lin', 'Hoang Tam Vo', 'Edward Guyot', 'Pierre Senellart', 'Guilherme Augusto Zagatti', 'Weidong Chen 0004']

-------------Graph: out-dblp_book--------------
The author with most collaborators is Sherif Sakr, with 10 collaborators:
['Hamid Reza Motahari-Nezhad', 'Daniela Grigori', 'Zuhair Khayyat', 'Seung Hwan Ryu', 'Seyed-Mehdi-Reza Beheshti', 'Faisal Moeen Orakzai', 'Boualem Benatallah', 'Moshe Chai Barukh', 'Ibrahim Abdelaziz', 'Ahmed Gater']

-------------Graph: out-dblp_incollection--------------
The author with most collaborators is Tuncer I. Ören, with 3

In [46]:
def get_largest_connected_component(G:nx.Graph)->nx.Graph:
    return G.subgraph(
    sorted(nx.connected_components(G), key = len, reverse=True)[0]
    ).copy()
    
def find_farther_node(G,starting_node:str)->list:
    edges = nx.bfs_edges(G,starting_node)
    edges = [starting_node] + [v for u, v in edges]
    return list(G.edges(edges[-1]))[0][0]

def two_sweep_path(G:nx.Graph,starting_node:str)->list:
    a = find_farther_node(G,starting_node)
    b = find_farther_node(G,a)
    return nx.shortest_path(G,a,b)

def calculate_starting_node(G: nx.Graph, method: str = "random") -> str:
    # G = get_largest_connected_component(G)

    random_node = choice(list(G.nodes))
    if method == "random":
        return random_node
    elif method == "2-sweep":
        starting_node = two_sweep_path(G, random_node)
        median_idx = len(starting_node) // 2
        starting_node = starting_node[median_idx]
    else:
        raise ValueError("Metodo non valido. Usare 'random' o '2-sweep'.")

    return starting_node

In [47]:
def calculate_F(G:nx.Graph,node:str,distance:int)->set:
    return nx.descendants_at_distance(G,node,distance)

def calculate_B_i(G:nx.Graph, node:str, i:int)->int:
    F = calculate_F(G,node,distance=i)
    B_i = 0
    for node in F:
        max = nx.eccentricity(G, v=node)
        if max > B_i:
            B_i = max
    return B_i


In [48]:
# Calcolo del Diametro
def iFub(G:nx.Graph, start_method:str = "random")-> int:
    G = get_largest_connected_component(G)
    node = calculate_starting_node(G,method=start_method)
    i = nx.eccentricity(G,v=node)

    lb = i
    ub = 2*lb
    
    while ub > lb:
        B_i = calculate_B_i(G,node,i)
        max = np.max([lb,B_i])
        if max > 2*(i-1):
            return max
        else:
            lb = max
            ub = 2*(i-1)
        i=i-1
    return lb

for idx, G in enumerate(graph_list):
    diameter = iFub(G)
    nx_diameter = nx.diameter(get_largest_connected_component(G))
    print(f"Graph {df_list[idx].name} has diameter: {diameter}")
    print(f"NetworkX diameter is {nx_diameter}\n")



Graph out-dblp_article has diameter: 7
NetworkX diameter is 7

Graph out-dblp_book has diameter: 4
NetworkX diameter is 4

Graph out-dblp_incollection has diameter: 7
NetworkX diameter is 7

Graph out-dblp_inproceedings has diameter: 16
NetworkX diameter is 16

Graph out-dblp_mastersthesis has diameter: 1
NetworkX diameter is 1

Graph out-dblp_phdthesis has diameter: 1
NetworkX diameter is 1

Graph out-dblp_proceedings has diameter: 30
NetworkX diameter is 30



In [49]:
def concatenate_DataFrame_from_list(df_list:list[pd.DataFrame])->pd.DataFrame:
    df = pd.concat(
        df_list,
        axis=0,
        ignore_index=True
    )
    df.drop_duplicates(
        subset='title',
        keep='first',
        inplace=True
    )
    return df

def build_union_graph_from_DataFrame_list(df_list:list[pd.DataFrame])->nx.Graph:
    return create_graph(
        concatenate_DataFrame_from_list(df_list)
    )



In [50]:
G = build_union_graph_from_DataFrame_list(df_list)

In [51]:
def answer_to_all_main_questions(G:nx.Graph,name:str)->None:
    print(f"-------------Graph: {name}--------------\n")

    title, authors, counter = get_publication_with_max_authors(G)
    print("The publication with most authors is:")
    print(f"{title} \n wich has {counter} authors\n")

    diameter = iFub(G)
    #nx_diameter = nx.diameter(get_largest_connected_component(G))
    print(f"The graph has exact diameter: {diameter}")
    #print(f"Diameter calculated with NetworkX is: {nx_diameter}\n")

    author, count, collaborators = get_author_with_most_collaborotors(G)
    print(f"The author with most collaborators is {author}, with {count} collaborators")

In [52]:
answer_to_all_main_questions(
    G,
    "Union"
)

-------------Graph: Union--------------

The publication with most authors is:
R-GMA: An Information Integration System for Grid Monitoring. 
 wich has 21 authors

The graph has exact diameter: 42
The author with most collaborators is Tuncer I. Ören, with 37 collaborators


In [53]:
import itertools

def build_authors_graph_from_DataFrame_list(df_list:list[pd.DataFrame])->nx.Graph:
    df = concatenate_DataFrame_from_list(df_list)
    df = df["author"]
    G = nx.Graph()
    for authors in df:
        authors_list = authors.split("|")


        
        for author_comb in itertools.combinations(authors_list,2):
            if G.has_edge(*author_comb):
                G[author_comb[0]][author_comb[1]]["weight"] += 1
            else:
                G.add_edge(*author_comb,weight = 1)
    return G

def find_most_collaborating_couple(G:nx.Graph)->tuple[str,str,dict[int]]:
    return  sorted(G.edges(data=True),key= lambda x: x[2]['weight'],reverse=True)[0]

author_1, author_2, weight = find_most_collaborating_couple(
    build_authors_graph_from_DataFrame_list(df_list)
    )

print(f"The most collaborating authors are {author_1} and {author_2} with {weight['weight']} collaborations togheter ")

The most collaborating authors are José D. P. Rolim and Klaus Jansen with 15 collaborations togheter 
